In [31]:
:dep plotters = {version = "0.3.5"}
:dep polars = {version = "0.35.0", features = ["describe", "lazy", "ndarray"]}
:dep ndarray = {version = "0.15.6"}
:dep smartcore = {version = "0.3.2"}

In [32]:
:show_deps

ndarray = {version = "0.15.6"}
plotters = {version = "0.3.5"}
polars = {version = "0.35.0", features = ["describe", "lazy", "ndarray"]}
smartcore = {version = "0.3.2"}


In [33]:
use polars::prelude::*;
use polars::frame::DataFrame;
use std::path::Path;
use ndarray::{ArrayBase, DataMut, Dimension, concatenate, Axis};
use plotters::prelude::*
use smartcore::linalg::basic::matrix::DenseMatrix;
use ndarray::prelude::*;

In [41]:
:doc DataFrame

polars_core::frame

pub struct DataFrame {
    pub(crate) columns: Vec<Series, Global>,
}


A contiguous growable collection of Series that have the same length.

Use declarationsAll the common tools can be found in crate::prelude (or in polars::prelude).

use polars_core::prelude::*; // if the crate polars-core is used directly
// use polars::prelude::*;      if the crate polars is used

InitializationDefaultA DataFrame can be initialized empty:

# use polars_core::prelude::*;
let df = DataFrame::default();
assert!(df.is_empty());

Wrapping a Vec<Series>A DataFrame is built upon a Vec<Series> where the Series have the same length.

# use polars_core::prelude::*;
let s1 = Series::new("Fruit", &["Apple", "Apple", "Pear"]);
let s2 = Series::new("Color", &["Red", "Yellow", "Green"]);

let df: PolarsResult<DataFrame> = DataFrame::new(vec![s1, s2]);

Using a macroThe df macro is a convenient method:

# use polars_core::prelude::*;
let df: PolarsResult<DataFrame> = df!("Fruit" => &["Apple", "Apple", "Pear"],
                                      "Color" => &["Red", "Yellow", "Green"]);

Using a CSV fileSee the polars_io::csv::CsvReader.

IndexingBy a numberThe Index<usize> is implemented for the DataFrame.

# use polars_core::prelude::*;
let df = df!("Fruit" => &["Apple", "Apple", "Pear"],
             "Color" => &["Red", "Yellow", "Green"])?;

assert_eq!(df[0], Series::new("Fruit", &["Apple", "Apple", "Pear"]));
assert_eq!(df[1], Series::new("Color", &["Red", "Yellow", "Green"]));
# Ok::<(), PolarsError>(())

By a Series name# use polars_core::prelude::*;
let df = df!("Fruit" => &["Apple", "Apple", "Pear"],
             "Color" => &["Red", "Yellow", "Green"])?;

assert_eq!(df["Fruit"], Series::new("Fruit", &["Apple", "Apple", "Pear"]));
assert_eq!(df["Color"], Series::new("Color", &["Red", "Yellow", "Green"]));
# Ok::<(), PolarsError>(())

In [42]:
//load dataset
fn read_data_frame_from_csv(
    csv_file_path: &Path,
) -> DataFrame{
    CsvReader::from_path(csv_file_path)
    .expect("Connot open file.")
    .has_header(true)
    .finish()
    .unwrap()
}

let iris_file_path: &Path = Path::new("C:\\Users\\PY245AG\\Downloads\\Iris.csv");
let iris_df: DataFrame = read_data_frame_from_csv(iris_file_path);
iris_df

shape: (150, 6)
┌─────┬───────────────┬──────────────┬───────────────┬──────────────┬────────────────┐
│ Id  ┆ SepalLengthCm ┆ SepalWidthCm ┆ PetalLengthCm ┆ PetalWidthCm ┆ Species        │
│ --- ┆ ---           ┆ ---          ┆ ---           ┆ ---          ┆ ---            │
│ i64 ┆ f64           ┆ f64          ┆ f64           ┆ f64          ┆ str            │
╞═════╪═══════════════╪══════════════╪═══════════════╪══════════════╪════════════════╡
│ 1   ┆ 5.1           ┆ 3.5          ┆ 1.4           ┆ 0.2          ┆ Iris-setosa    │
│ 2   ┆ 4.9           ┆ 3.0          ┆ 1.4           ┆ 0.2          ┆ Iris-setosa    │
│ 3   ┆ 4.7           ┆ 3.2          ┆ 1.3           ┆ 0.2          ┆ Iris-setosa    │
│ 4   ┆ 4.6           ┆ 3.1          ┆ 1.5           ┆ 0.2          ┆ Iris-setosa    │
│ …   ┆ …             ┆ …            ┆ …             ┆ …            ┆ …              │
│ 147 ┆ 6.3           ┆ 2.5          ┆ 5.0           ┆ 1.9          ┆ Iris-virginica │
│ 148 ┆ 6.5           ┆ 3.0

In [43]:
:type iris_df

iris_df: DataFrame

In [44]:
iris_df.shape()

(150, 6)

In [45]:
iris_df.head(Some(5))

shape: (5, 6)
┌─────┬───────────────┬──────────────┬───────────────┬──────────────┬─────────────┐
│ Id  ┆ SepalLengthCm ┆ SepalWidthCm ┆ PetalLengthCm ┆ PetalWidthCm ┆ Species     │
│ --- ┆ ---           ┆ ---          ┆ ---           ┆ ---          ┆ ---         │
│ i64 ┆ f64           ┆ f64          ┆ f64           ┆ f64          ┆ str         │
╞═════╪═══════════════╪══════════════╪═══════════════╪══════════════╪═════════════╡
│ 1   ┆ 5.1           ┆ 3.5          ┆ 1.4           ┆ 0.2          ┆ Iris-setosa │
│ 2   ┆ 4.9           ┆ 3.0          ┆ 1.4           ┆ 0.2          ┆ Iris-setosa │
│ 3   ┆ 4.7           ┆ 3.2          ┆ 1.3           ┆ 0.2          ┆ Iris-setosa │
│ 4   ┆ 4.6           ┆ 3.1          ┆ 1.5           ┆ 0.2          ┆ Iris-setosa │
│ 5   ┆ 5.0           ┆ 3.6          ┆ 1.4           ┆ 0.2          ┆ Iris-setosa │
└─────┴───────────────┴──────────────┴───────────────┴──────────────┴─────────────┘

In [46]:
iris_df.tail(Some(5))

shape: (5, 6)
┌─────┬───────────────┬──────────────┬───────────────┬──────────────┬────────────────┐
│ Id  ┆ SepalLengthCm ┆ SepalWidthCm ┆ PetalLengthCm ┆ PetalWidthCm ┆ Species        │
│ --- ┆ ---           ┆ ---          ┆ ---           ┆ ---          ┆ ---            │
│ i64 ┆ f64           ┆ f64          ┆ f64           ┆ f64          ┆ str            │
╞═════╪═══════════════╪══════════════╪═══════════════╪══════════════╪════════════════╡
│ 146 ┆ 6.7           ┆ 3.0          ┆ 5.2           ┆ 2.3          ┆ Iris-virginica │
│ 147 ┆ 6.3           ┆ 2.5          ┆ 5.0           ┆ 1.9          ┆ Iris-virginica │
│ 148 ┆ 6.5           ┆ 3.0          ┆ 5.2           ┆ 2.0          ┆ Iris-virginica │
│ 149 ┆ 6.2           ┆ 3.4          ┆ 5.4           ┆ 2.3          ┆ Iris-virginica │
│ 150 ┆ 5.9           ┆ 3.0          ┆ 5.1           ┆ 1.8          ┆ Iris-virginica │
└─────┴───────────────┴──────────────┴───────────────┴──────────────┴────────────────┘

In [49]:
iris_df.describe(None)?

shape: (9, 7)
┌────────────┬───────────┬──────────────┬──────────────┬──────────────┬──────────────┬─────────────┐
│ describe   ┆ Id        ┆ SepalLengthC ┆ SepalWidthCm ┆ PetalLengthC ┆ PetalWidthCm ┆ Species     │
│ ---        ┆ ---       ┆ m            ┆ ---          ┆ m            ┆ ---          ┆ ---         │
│ str        ┆ f64       ┆ ---          ┆ f64          ┆ ---          ┆ f64          ┆ str         │
│            ┆           ┆ f64          ┆              ┆ f64          ┆              ┆             │
╞════════════╪═══════════╪══════════════╪══════════════╪══════════════╪══════════════╪═════════════╡
│ count      ┆ 150.0     ┆ 150.0        ┆ 150.0        ┆ 150.0        ┆ 150.0        ┆ 150         │
│ null_count ┆ 0.0       ┆ 0.0          ┆ 0.0          ┆ 0.0          ┆ 0.0          ┆ 0           │
│ mean       ┆ 75.5      ┆ 5.843333     ┆ 3.054        ┆ 3.758667     ┆ 1.198667     ┆ null        │
│ std        ┆ 43.445368 ┆ 0.828066     ┆ 0.433594     ┆ 1.76442      ┆ 0.763

In [50]:
let column_names = iris_df.get_column_names();
{
    column_names
}

["Id", "SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm", "Species"]

In [52]:
let numeric_iris_df: DataFrame = iris_df.drop("Species")?;

In [53]:
numeric_iris_df.mean()

shape: (1, 5)
┌──────┬───────────────┬──────────────┬───────────────┬──────────────┐
│ Id   ┆ SepalLengthCm ┆ SepalWidthCm ┆ PetalLengthCm ┆ PetalWidthCm │
│ ---  ┆ ---           ┆ ---          ┆ ---           ┆ ---          │
│ f64  ┆ f64           ┆ f64          ┆ f64           ┆ f64          │
╞══════╪═══════════════╪══════════════╪═══════════════╪══════════════╡
│ 75.5 ┆ 5.843333      ┆ 3.054        ┆ 3.758667      ┆ 1.198667     │
└──────┴───────────────┴──────────────┴───────────────┴──────────────┘

In [54]:
numeric_iris_df.max()

shape: (1, 5)
┌─────┬───────────────┬──────────────┬───────────────┬──────────────┐
│ Id  ┆ SepalLengthCm ┆ SepalWidthCm ┆ PetalLengthCm ┆ PetalWidthCm │
│ --- ┆ ---           ┆ ---          ┆ ---           ┆ ---          │
│ i64 ┆ f64           ┆ f64          ┆ f64           ┆ f64          │
╞═════╪═══════════════╪══════════════╪═══════════════╪══════════════╡
│ 150 ┆ 7.9           ┆ 4.4          ┆ 6.9           ┆ 2.5          │
└─────┴───────────────┴──────────────┴───────────────┴──────────────┘

In [56]:
// convert to ndarray
let numeric_iris_ndarray: ArrayBase<_, _> = numeric_iris_df.to_ndarray::<Float64Type>(IndexOrder::Fortran).unwrap();
numeric_iris_ndarray

[[1.0, 5.1, 3.5, 1.4, 0.2],
 [2.0, 4.9, 3.0, 1.4, 0.2],
 [3.0, 4.7, 3.2, 1.3, 0.2],
 [4.0, 4.6, 3.1, 1.5, 0.2],
 [5.0, 5.0, 3.6, 1.4, 0.2],
 ...,
 [146.0, 6.7, 3.0, 5.2, 2.3],
 [147.0, 6.3, 2.5, 5.0, 1.9],
 [148.0, 6.5, 3.0, 5.2, 2.0],
 [149.0, 6.2, 3.4, 5.4, 2.3],
 [150.0, 5.9, 3.0, 5.1, 1.8]], shape=[150, 5], strides=[1, 150], layout=Ff (0xa), const ndim=2

In [57]:
//ndarray: first element
numeric_iris_ndarray[[0, 0]]

1.0

In [61]:
//ndarray: first row
numeric_iris_ndarray.slice(s![0, ..])

[1.0, 5.1, 3.5, 1.4, 0.2], shape=[5], strides=[150], layout=Custom (0x0), const ndim=1

In [62]:
//ndarray: shape
numeric_iris_ndarray.dim()

(150, 5)

In [64]:
//ndarray: last row
let numeric_iris_ndarray_shape = numeric_iris_ndarray.dim();
numeric_iris_ndarray.slice(s![-1, ..])

[150.0, 5.9, 3.0, 5.1, 1.8], shape=[5], strides=[150], layout=Custom (0x0), const ndim=1

In [65]:
//ndarray: first five row
numeric_iris_ndarray.slice(s![0..5, ..])

[[1.0, 5.1, 3.5, 1.4, 0.2],
 [2.0, 4.9, 3.0, 1.4, 0.2],
 [3.0, 4.7, 3.2, 1.3, 0.2],
 [4.0, 4.6, 3.1, 1.5, 0.2],
 [5.0, 5.0, 3.6, 1.4, 0.2]], shape=[5, 5], strides=[1, 150], layout=f (0x8), const ndim=2

In [68]:
//ndarray: last five row
numeric_iris_ndarray.slice(s![-5.., ..])

[[146.0, 6.7, 3.0, 5.2, 2.3],
 [147.0, 6.3, 2.5, 5.0, 1.9],
 [148.0, 6.5, 3.0, 5.2, 2.0],
 [149.0, 6.2, 3.4, 5.4, 2.3],
 [150.0, 5.9, 3.0, 5.1, 1.8]], shape=[5, 5], strides=[1, 150], layout=f (0x8), const ndim=2

In [69]:
//check for zero elements
numeric_iris_ndarray.is_empty()

false